In [11]:
from src.evaluator.MLP_baseline_evaluator import MLPBaselineEvaluator
from src.models.MLP_concat import MLPModel
from src.utils import *
from src.evaluator.evaluator_utils import l2_loss
from src.dataset.dataset_unseen_compounds import SciplexDatasetUnseenPerturbations
from sklearn.metrics import r2_score

import anndata as ad
import math
import pickle as pkl
import optuna

In [2]:
#read PRNet splits
with open("./data/sciplex/prnet_drug_splits.pkl", "rb") as f:
    drug_splits = pkl.load(f)

In [3]:
drugs_train = drug_splits['drug_split_0']['train']
drugs_validation = drug_splits['drug_split_0']['valid']
drugs_test = drug_splits['drug_split_0']['test']

In [4]:
ad_path = "./data/sciplex/sciplex_final.h5ad"
adata = ad.read_h5ad(ad_path)

dataset_train = SciplexDatasetUnseenPerturbations(adata, drugs_train, "sm_coati_emb", 256, "X_2000_hvg", "X_2000_hvg")
dataset_validation = SciplexDatasetUnseenPerturbations(adata, drugs_validation, "sm_coati_emb", 256, "X_2000_hvg", "X_2000_hvg")
dataset_test = SciplexDatasetUnseenPerturbations(adata, drugs_test, "sm_coati_emb", 256, "X_2000_hvg", "X_2000_hvg")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 571696/571696 [00:32<00:00, 17574.41it/s]


In [14]:
def objective(trial, dataset_train=None, dataset_validation=None, dataset_test=None,
              input_dim=0, output_dim=0, drug_dim=0, scheduler_mode='min', loss_fn=None):
    # #static params
    # input_dim = 2000
    # output_dim = 2000
    # drug_dim = 256
    # loss_fn = l2_loss
    # scheduler_mode = "min"

    #dynamic params
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
    scheduler_factor = trial.suggest_loguniform('scheduler_factor', 0.1, 0.5)
    scheduler_patience = trial.suggest_int('scheduler_patience', 1, 20)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128, 256])
    dropout = trial.suggest_loguniform('dropout', 0.05, 0.3)
    hidden_dims = trial.suggest_categorical('hidden_dims', [[64], [128], [256], [512], [1024], [2048], [4096]])

    params = {
        'input_dim': input_dim,
        'output_dim' : output_dim,
        'drug_dim' : drug_dim,
        'dropout' : dropout,
        'scheduler_mode': scheduler_mode,
        'lr': lr,
        'weight_decay': weight_decay,
        'scheduler_factor': scheduler_factor,
        'scheduler_patience': scheduler_patience,
        'batch_size': batch_size,
        'dropout': dropout,
        'hidden_dims' : hidden_dims,
    }
    ev = MLPBaselineEvaluator(dataset_train, dataset_validation, dataset_test, params)

    return ev.train(loss_fn, trial)

In [16]:
def cross_validation_models(drug_splits, loss_function, adata, input_name, input_dim, output_name, output_dim, drug_rep_name, drug_emb_size):
    output = dict()
    for i in range(5):
        drugs_train = drug_splits[f'drug_split_{i}']['train']
        drugs_validation = drug_splits[f'drug_split_{i}']['valid']
        drugs_test = drug_splits[f'drug_split_{i}']['test']

        dataset_train = SciplexDatasetUnseenPerturbations(adata, drugs_train, drug_rep_name, drug_emb_size, input_name, output_name)
        dataset_validation = SciplexDatasetUnseenPerturbations(adata, drugs_validation, drug_rep_name, drug_emb_size, input_name, output_name)
        dataset_test = SciplexDatasetUnseenPerturbations(adata, drugs_test, drug_rep_name, drug_emb_size, input_name, output_name)

        study = optuna.create_study(direction='minimize')
        study.optimize(lambda trial: objective(trial, 
                                               dataset_train=dataset_train, dataset_validation=dataset_validation, dataset_test=dataset_test,
                                               input_dim=input_dim, output_dim=output_dim, 
                                               drug_dim=drug_emb_size, loss_fn=loss_function), n_trials=50)
        best_params = study.best_params

        output[i] = dict()
        output[i]['best_params'] = best_params

    return output

In [17]:
with open("./data/sciplex/prnet_drug_splits.pkl", "rb") as f:
    drug_splits = pkl.load(f)

ad_path = "./data/sciplex/sciplex_final.h5ad"
adata = ad.read_h5ad(ad_path)

mlp_params_drugsplit = cross_validation_models(drug_splits, l2_loss, adata, "X_2000_hvg", 2000, "X_2000_hvg", 2000, "sm_coati_emb", 256)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 571696/571696 [00:32<00:00, 17739.78it/s]
[I 2025-04-01 16:05:21,721] A new study created in memory with name: no-name-fffe5b28-8e91-47a4-8792-c3c773a67e19
/tmp/ipykernel_11479/3753511677.py:11: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
/tmp/ipykernel_11479/3753511677.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-2)
/tmp/ipykernel_11479/3753511677.py:13: FutureWarning: suggest_loguniform has been deprecated in v

Early stopping at epoch 78


[I 2025-04-01 16:07:27,078] Trial 1 finished with value: 0.029305404074853512 and parameters: {'lr': 0.0026261830362890037, 'weight_decay': 2.413078974070546e-05, 'scheduler_factor': 0.11666377168699496, 'scheduler_patience': 16, 'batch_size': 128, 'dropout': 0.06652978720814749, 'hidden_dims': [2048]}. Best is trial 0 with value: 0.023288911390231877.


Early stopping at epoch 11


[I 2025-04-01 16:08:22,023] Trial 2 finished with value: 0.02326872913939197 and parameters: {'lr': 0.0004047317489266281, 'weight_decay': 3.0750066589846114e-06, 'scheduler_factor': 0.24726233063880135, 'scheduler_patience': 4, 'batch_size': 128, 'dropout': 0.09786634141917266, 'hidden_dims': [2048]}. Best is trial 2 with value: 0.02326872913939197.


...

[I 2025-04-01 19:04:41,696] Trial 33 finished with value: 0.023934129089534486 and parameters: {'lr': 0.00034632120737260424, 'weight_decay': 1.5414521474163596e-06, 'scheduler_factor': 0.40220814227456986, 'scheduler_patience': 13, 'batch_size': 16, 'dropout': 0.14286073936138835, 'hidden_dims': [256]}. Best is trial 5 with value: 0.023913027440900344.


Early stopping at epoch 21


[I 2025-04-01 19:04:50,981] Trial 34 pruned. 
[I 2025-04-01 19:04:56,834] Trial 35 pruned. 
[I 2025-04-01 19:05:06,240] Trial 36 pruned. 
[I 2025-04-01 19:05:09,250] Trial 37 pruned. 
[I 2025-04-01 19:05:10,739] Trial 38 pruned. 
[I 2025-04-01 19:05:20,485] Trial 39 pruned. 
[I 2025-04-01 19:05:24,359] Trial 40 pruned. 
[I 2025-04-01 19:07:38,665] Trial 41 finished with value: 0.02391581031734711 and parameters: {'lr': 0.0003011919330062834, 'weight_decay': 1.051991214075546e-06, 'scheduler_factor': 0.3985942480779108, 'scheduler_patience': 13, 'batch_size': 16, 'dropout': 0.13977338053249536, 'hidden_dims': [256]}. Best is trial 5 with value: 0.023913027440900344.


Early stopping at epoch 27


[I 2025-04-01 19:07:48,141] Trial 42 pruned. 
[I 2025-04-01 19:07:57,729] Trial 43 pruned. 
[I 2025-04-01 19:08:02,944] Trial 44 pruned. 
[I 2025-04-01 19:08:06,104] Trial 45 pruned. 
[I 2025-04-01 19:08:07,258] Trial 46 pruned. 
[I 2025-04-01 19:08:23,924] Trial 47 pruned. 
[I 2025-04-01 19:08:28,678] Trial 48 pruned. 
[I 2025-04-01 19:08:30,111] Trial 49 pruned. 


In [18]:
print(mlp_params_drugsplit)

{0: {'best_params': {'lr': 0.0008596644539487043, 'weight_decay': 6.953857338614077e-06, 'scheduler_factor': 0.10468758349886369, 'scheduler_patience': 8, 'batch_size': 128, 'dropout': 0.12003473622754349, 'hidden_dims': [64]}}, 1: {'best_params': {'lr': 0.0004543426062151711, 'weight_decay': 9.240565393725398e-06, 'scheduler_factor': 0.1086972953285817, 'scheduler_patience': 6, 'batch_size': 32, 'dropout': 0.07473484554256053, 'hidden_dims': [128]}}, 2: {'best_params': {'lr': 3.301745485905233e-05, 'weight_decay': 4.338830329161768e-06, 'scheduler_factor': 0.28105924419350814, 'scheduler_patience': 13, 'batch_size': 16, 'dropout': 0.07653937722833731, 'hidden_dims': [1024]}}, 3: {'best_params': {'lr': 0.00054584005358527, 'weight_decay': 3.968393385665429e-06, 'scheduler_factor': 0.1581167403793244, 'scheduler_patience': 5, 'batch_size': 32, 'dropout': 0.27022795713463577, 'hidden_dims': [256]}}, 4: {'best_params': {'lr': 7.179753439362002e-05, 'weight_decay': 1.6647715168053354e-05, 

In [ ]:
best_params_2000HVG = {
    "lr": 0.0001,
    "weight_decay": 
}

In [10]:
adata_control = ad.read_h5ad("../data/sciplex/sciplex_final.h5ad")
gene_names = adata_control.uns['gene_names_2000']
adata_control = adata_control[adata_control.obs['product_name'] == "Vehicle"]

In [11]:
performance = get_model_stats(results, adata_control, "X_5000_hvg", gene_names, "test")

/home/victor/.local/lib/python3.12/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [12]:
performance

{'key': 'test',
 'mse_A549': np.float32(0.0027460095),
 'std_mse_A549': np.float32(0.0006309031),
 'mse_K562': np.float32(0.0026446814),
 'std_mse_K562': np.float32(0.0008933902),
 'mse_MCF7': np.float32(0.001277654),
 'std_mse_MCF7': np.float32(0.00012553648),
 'r2_A549': np.float64(0.8523319363594055),
 'std_r2_A549': np.float64(0.015389074220604145),
 'r2_K562': np.float64(0.7614692714479234),
 'std_r2_K562': np.float64(0.06679660095799952),
 'r2_MCF7': np.float64(0.7426602376831902),
 'std_r2_MCF7': np.float64(0.015389074220604145),
 'rank_logfc_A549': np.float64(0.49444444444444435),
 'std_rank_logfc_A549': np.float64(0.29888829881336687),
 'rank_logfc_K562': np.float64(0.5015873015873017),
 'std_rank_logfc_K562': np.float64(0.2899248823225953),
 'rank_logfc_MCF7': np.float64(0.4999999999999999),
 'std_rank_logfc_MCF7': np.float64(0.2938137685886162)}